## NEW work and API

In [1]:
import json
with open("credentials.json",'r') as f:
    credentials = json.load(f)

In [10]:
from openaq import OpenAQ

client = OpenAQ(api_key=credentials['OPENAQ-API-KEY'])
ls = client.locations.list(parameters_id=2, limit=1000)
client.close()

RuntimeError: Cannot send a request, as the client has been closed.

In [3]:
from openaq import OpenAQ

client = OpenAQ(api_key=credentials['OPENAQ-API-KEY'])

# location = client.locations.get(2178)

# client.close()

# print(location)

In [4]:
LocationsResponse = client.locations.list(radius=5000, coordinates=(12.964603032123327, 77.67439755092434))
LocationsResponse

LocationsResponse(headers=Headers(x_ratelimit_limit=60, x_ratelimit_remaining=59, x_ratelimit_used=1, x_ratelimit_reset=60), meta=Meta(name='openaq-api', website='/', page=1, limit=100, found=3), results=[Location(id=2592, name='BWSSB Kadabesanahalli, Bengaluru - KSPCB', locality=None, timezone='Asia/Kolkata', country=CountryBase(id=9, code='IN', name='India'), owner=OwnerBase(id=4, name='Unknown Governmental Organization'), provider=ProviderBase(id=168, name='CPCB'), is_mobile=False, is_monitor=True, instruments=[InstrumentBase(id=2, name='Government Monitor')], sensors=[SensorBase(id=5381, name='co µg/m³', parameter=ParameterBase(id=4, name='co', units='µg/m³', display_name='CO mass')), SensorBase(id=5380, name='no2 µg/m³', parameter=ParameterBase(id=5, name='no2', units='µg/m³', display_name='NO₂ mass')), SensorBase(id=5375, name='o3 µg/m³', parameter=ParameterBase(id=3, name='o3', units='µg/m³', display_name='O₃ mass')), SensorBase(id=13898, name='pm25 µg/m³', parameter=ParameterBa

In [5]:
LocationsResponse.results[0].name

'BWSSB Kadabesanahalli, Bengaluru - KSPCB'

In [7]:
[f.id for i in range(len(LocationsResponse.results)) for f in LocationsResponse.results[i].sensors]

[5381,
 5380,
 5375,
 13898,
 13888,
 14637,
 14636,
 14640,
 1303004,
 14641,
 14634,
 12237595,
 12237596,
 12237597,
 12237598,
 12237599,
 12237600,
 12237601,
 12237602,
 12237603]

In [12]:
LocationsResponse.results[0].sensors

[SensorBase(id=5381, name='co µg/m³', parameter=ParameterBase(id=4, name='co', units='µg/m³', display_name='CO mass')),
 SensorBase(id=5380, name='no2 µg/m³', parameter=ParameterBase(id=5, name='no2', units='µg/m³', display_name='NO₂ mass')),
 SensorBase(id=5375, name='o3 µg/m³', parameter=ParameterBase(id=3, name='o3', units='µg/m³', display_name='O₃ mass')),
 SensorBase(id=13898, name='pm25 µg/m³', parameter=ParameterBase(id=2, name='pm25', units='µg/m³', display_name='PM2.5')),
 SensorBase(id=13888, name='so2 µg/m³', parameter=ParameterBase(id=6, name='so2', units='µg/m³', display_name='SO₂ mass'))]

In [13]:
readings = client.measurements.list(sensors_id=5381, limit=1000)

In [14]:
readings.meta

Meta(name='openaq-api', website='/', page=1, limit=1000, found='>1000')

In [15]:
readings.results

[Measurement(period=Period(label='raw', interval='01:00:00', datetime_from=Datetime(utc='2016-10-09T17:45:00Z', local='2016-10-09T23:15:00+05:30'), datetime_to=Datetime(utc='2016-10-09T18:45:00Z', local='2016-10-10T00:15:00+05:30')), value=600.0, parameter=ParameterBase(id=4, name='co', units='µg/m³', display_name=None), coordinates=None, summary=None, coverage=Coverage(expected_count=1, expected_interval='01:00:00', observed_count=1, observed_interval='00:15:00', percent_complete=100.0, percent_coverage=0.0, datetime_from=Datetime(utc='2016-10-09T18:30:00Z', local='2016-10-10T00:00:00+05:30'), datetime_to=Datetime(utc='2016-10-09T18:45:00Z', local='2016-10-10T00:15:00+05:30'))),
 Measurement(period=Period(label='raw', interval='01:00:00', datetime_from=Datetime(utc='2016-10-09T18:15:00Z', local='2016-10-09T23:45:00+05:30'), datetime_to=Datetime(utc='2016-10-09T19:15:00Z', local='2016-10-10T00:45:00+05:30')), value=320.0, parameter=ParameterBase(id=4, name='co', units='µg/m³', display_

In [17]:
readings.results[:2]

[Measurement(period=Period(label='raw', interval='01:00:00', datetime_from=Datetime(utc='2016-10-09T17:45:00Z', local='2016-10-09T23:15:00+05:30'), datetime_to=Datetime(utc='2016-10-09T18:45:00Z', local='2016-10-10T00:15:00+05:30')), value=600.0, parameter=ParameterBase(id=4, name='co', units='µg/m³', display_name=None), coordinates=None, summary=None, coverage=Coverage(expected_count=1, expected_interval='01:00:00', observed_count=1, observed_interval='00:15:00', percent_complete=100.0, percent_coverage=0.0, datetime_from=Datetime(utc='2016-10-09T18:30:00Z', local='2016-10-10T00:00:00+05:30'), datetime_to=Datetime(utc='2016-10-09T18:45:00Z', local='2016-10-10T00:15:00+05:30'))),
 Measurement(period=Period(label='raw', interval='01:00:00', datetime_from=Datetime(utc='2016-10-09T18:15:00Z', local='2016-10-09T23:45:00+05:30'), datetime_to=Datetime(utc='2016-10-09T19:15:00Z', local='2016-10-10T00:45:00+05:30')), value=320.0, parameter=ParameterBase(id=4, name='co', units='µg/m³', display_

In [36]:
import json
with open("response_1756734808678.json",'r') as f:
    res = json.load(f)

In [37]:
len(res['results'])

21

In [42]:
res['results'][0]['name']

'Peenya, Bengaluru - KSPCB'

In [49]:
res['results'][0]['sensors']

[{'id': 722,
  'name': 'co Âµg/mÂ³',
  'parameter': {'id': 4,
   'name': 'co',
   'units': 'Âµg/mÂ³',
   'displayName': 'CO mass'}},
 {'id': 1518,
  'name': 'no2 Âµg/mÂ³',
  'parameter': {'id': 5,
   'name': 'no2',
   'units': 'Âµg/mÂ³',
   'displayName': 'NOâ‚‚ mass'}},
 {'id': 13852,
  'name': 'pm25 Âµg/mÂ³',
  'parameter': {'id': 2,
   'name': 'pm25',
   'units': 'Âµg/mÂ³',
   'displayName': 'PM2.5'}},
 {'id': 13853,
  'name': 'so2 Âµg/mÂ³',
  'parameter': {'id': 6,
   'name': 'so2',
   'units': 'Âµg/mÂ³',
   'displayName': 'SOâ‚‚ mass'}}]

In [51]:
[f['id'] for i in range(len(res['results'])) for f in res['results'][i]['sensors']]

[722,
 1518,
 13852,
 13853,
 1048,
 1427,
 4886,
 13836,
 13837,
 1384,
 1383,
 1382,
 14096,
 5350,
 5349,
 5348,
 14129,
 5381,
 5380,
 5375,
 13898,
 13888,
 14637,
 14636,
 14640,
 1303004,
 14641,
 14634,
 14657,
 12235356,
 12235357,
 14639,
 12235358,
 14658,
 12235359,
 12235360,
 1305377,
 14635,
 12235361,
 12235362,
 14642,
 12235363,
 12235364,
 12235374,
 14779,
 12235375,
 14773,
 12235376,
 12235377,
 14775,
 14732,
 12235378,
 15270,
 12235365,
 12235366,
 12235367,
 14850,
 40537,
 12235368,
 12235369,
 1303005,
 12235370,
 14859,
 12235371,
 14862,
 12235372,
 12235373,
 15039,
 12235379,
 12235380,
 12235381,
 15035,
 15033,
 12235382,
 12235383,
 12235384,
 15032,
 12235385,
 20173,
 12235262,
 12235263,
 20175,
 12235264,
 12235265,
 20155,
 12235266,
 20172,
 12235267,
 20162,
 12235268,
 12235269,
 20152,
 12235270,
 12235235,
 20153,
 12235236,
 12235237,
 20169,
 12235238,
 20167,
 12235239,
 20166,
 12235240,
 20165,
 12235241,
 20164,
 12235242,
 12235243,
 

In [46]:
[f['sensors'] for f in res['results']]

[[{'id': 722,
   'name': 'co Âµg/mÂ³',
   'parameter': {'id': 4,
    'name': 'co',
    'units': 'Âµg/mÂ³',
    'displayName': 'CO mass'}},
  {'id': 1518,
   'name': 'no2 Âµg/mÂ³',
   'parameter': {'id': 5,
    'name': 'no2',
    'units': 'Âµg/mÂ³',
    'displayName': 'NOâ‚‚ mass'}},
  {'id': 13852,
   'name': 'pm25 Âµg/mÂ³',
   'parameter': {'id': 2,
    'name': 'pm25',
    'units': 'Âµg/mÂ³',
    'displayName': 'PM2.5'}},
  {'id': 13853,
   'name': 'so2 Âµg/mÂ³',
   'parameter': {'id': 6,
    'name': 'so2',
    'units': 'Âµg/mÂ³',
    'displayName': 'SOâ‚‚ mass'}}],
 [{'id': 1048,
   'name': 'co Âµg/mÂ³',
   'parameter': {'id': 4,
    'name': 'co',
    'units': 'Âµg/mÂ³',
    'displayName': 'CO mass'}},
  {'id': 1427,
   'name': 'no2 Âµg/mÂ³',
   'parameter': {'id': 5,
    'name': 'no2',
    'units': 'Âµg/mÂ³',
    'displayName': 'NOâ‚‚ mass'}},
  {'id': 4886,
   'name': 'o3 Âµg/mÂ³',
   'parameter': {'id': 3,
    'name': 'o3',
    'units': 'Âµg/mÂ³',
    'displayName': 'Oâ‚ƒ mass'}},

In [130]:
import time
import pandas as pd
from pathlib import Path
def get_data(sensor_id, datetime_from, datetime_to):
    data = []
    timestamps = []
    page=1
    limit=1000
    while True:
        readings = client.measurements.list(sensors_id=sensor_id, limit=limit, page=page, datetime_from=datetime_from, datetime_to=datetime_to)

        
        if readings.meta.found == 0:
            print(f"No readings found from {datetime_from} to {datetime_to}")
            return

        if page==1:
            pollutant_name = readings.results[0].parameter.name
            units = readings.results[0].parameter.units
            col_name = f"{pollutant_name}_{units.replace('/', '_').replace(' ', '')}"


        page_data = [f.value for f in readings.results]
        page_timestamps = [f.period.datetime_to.utc for f in readings.results]
        
        data.extend(page_data)
        timestamps.extend(page_timestamps)

        print(f"Fetched {len(page_data)} readings from page {page}. Total readings so far: {len(data)}")
        if len(page_data)<limit:
            print("Reached the end of data.")
            break

        page += 1
        time.sleep(5)
    
    print(f"Finished Fetching all data. Total readings are {len(data)}")
    if data:
        data_dict = {'Timestamp': timestamps,
                    col_name: data}
        data_df = pd.DataFrame(data_dict)
        data_file_path = Path("AQ_data") / str(sensor_id) / datetime_from[:4]
        data_file_path.mkdir(exist_ok=True, parents=True)
        file_name = data_file_path / (pollutant_name+'.csv')
        # print(type(file_name), file_name)
        data_df.to_csv(file_name, index=False)
        print(f"saved the data to {file_name}")

In [131]:
get_data(1382, datetime_from="2017-01-01", datetime_to="2017-12-31")

Fetched 1000 readings from page 1. Total readings so far: 1000
Fetched 1000 readings from page 2. Total readings so far: 2000
Fetched 1000 readings from page 3. Total readings so far: 3000
Fetched 1000 readings from page 4. Total readings so far: 4000
Fetched 1000 readings from page 5. Total readings so far: 5000
Fetched 1000 readings from page 6. Total readings so far: 6000
Fetched 1000 readings from page 7. Total readings so far: 7000
Fetched 1000 readings from page 8. Total readings so far: 8000
Fetched 1000 readings from page 9. Total readings so far: 9000
Fetched 1000 readings from page 10. Total readings so far: 10000
Fetched 1000 readings from page 11. Total readings so far: 11000
Fetched 1000 readings from page 12. Total readings so far: 12000
Fetched 467 readings from page 13. Total readings so far: 12467
Reached the end of data.
Finished Fetching all data. Total readings are 12467
saved the data to AQ_data\1382\2017\pm10.csv


In [105]:
import time
import pandas as pd
from pathlib import Path

def get_data_for_pollutants(sensor_ids, datetime_from, datetime_to):
    """
    Query multiple pollutants for a given list of sensor_ids and 
    return a single DataFrame with timestamp + pollutant(unit) columns.
    """

    all_pollutants_data = {}   # key = pollutant(unit), value = list of values
    timestamps = None          # global timestamp alignment

    for sensor_id in sensor_ids:
        print(f"Fetching data for sensor {sensor_id}...")
        data = []
        ts = []
        page = 1
        limit = 1000
        pollutant_name, units = None, None

        while True:
            readings = client.measurements.list(
                sensors_id=sensor_id,
                limit=limit,
                page=page,
                datetime_from=datetime_from,
                datetime_to=datetime_to
            )

            if page == 1:
                pollutant_name = readings.results[0].parameter.name
                units = readings.results[0].parameter.units
                col_name = f"{pollutant_name}_{units.replace('/', '_').replace(' ', '')}"

            page_data = [f.value for f in readings.results]
            page_timestamps = [f.period.datetime_to.utc for f in readings.results]

            data.extend(page_data)
            ts.extend(page_timestamps)

            print(f"Fetched {len(page_data)} rows from page {page}. Total so far: {len(data)}")

            if len(page_data) < limit:
                print("Reached the end of data.")
                break

            page += 1
            time.sleep(45)  # respect API rate limits

        if timestamps is None:
            timestamps = ts  # initialize timestamps from first pollutant
        else:
            if len(ts) != len(timestamps):
                print(f"⚠️ Warning: timestamps mismatch for {pollutant_name}")

        all_pollutants_data[col_name] = data

    # Build DataFrame with all pollutants
    df = pd.DataFrame({"Timestamp": timestamps})
    for pollutant_col, values in all_pollutants_data.items():
        df[pollutant_col] = values

    # Save to CSV
    data_file_path = Path("AQ_data") / datetime_from[:4]
    data_file_path.mkdir(exist_ok=True, parents=True)
    file_name = data_file_path / "city_pollutants.csv"
    df.to_csv(file_name, index=False)

    print(f"✅ Saved all pollutants to {file_name}")
    return df


In [80]:
pollutant_name

NameError: name 'pollutant_name' is not defined

In [67]:
readings.meta

Meta(name='openaq-api', website='/', page=1, limit=1000, found='>1000')

In [97]:
readings = client.measurements.list(sensors_id=14634, limit=1000)

In [103]:
readings

MeasurementsResponse(headers=Headers(x_ratelimit_limit=60, x_ratelimit_remaining=59, x_ratelimit_used=1, x_ratelimit_reset=60), meta=Meta(name='openaq-api', website='/', page=1, limit=1000, found='>1000'), results=[Measurement(period=Period(label='raw', interval='01:00:00', datetime_from=Datetime(utc='2018-03-10T18:30:00Z', local='2018-03-11T00:00:00+05:30'), datetime_to=Datetime(utc='2018-03-10T19:30:00Z', local='2018-03-11T01:00:00+05:30')), value=8.0, parameter=ParameterBase(id=6, name='so2', units='µg/m³', display_name=None), coordinates=None, summary=None, coverage=Coverage(expected_count=1, expected_interval='01:00:00', observed_count=1, observed_interval='01:00:00', percent_complete=100.0, percent_coverage=100.0, datetime_from=Datetime(utc='2018-03-10T18:30:00Z', local='2018-03-11T00:00:00+05:30'), datetime_to=Datetime(utc='2018-03-10T19:30:00Z', local='2018-03-11T01:00:00+05:30'))), Measurement(period=Period(label='raw', interval='01:00:00', datetime_from=Datetime(utc='2018-03-

In [101]:
readings.results

[Measurement(period=Period(label='raw', interval='01:00:00', datetime_from=Datetime(utc='2018-03-10T18:30:00Z', local='2018-03-11T00:00:00+05:30'), datetime_to=Datetime(utc='2018-03-10T19:30:00Z', local='2018-03-11T01:00:00+05:30')), value=8.0, parameter=ParameterBase(id=6, name='so2', units='µg/m³', display_name=None), coordinates=None, summary=None, coverage=Coverage(expected_count=1, expected_interval='01:00:00', observed_count=1, observed_interval='01:00:00', percent_complete=100.0, percent_coverage=100.0, datetime_from=Datetime(utc='2018-03-10T18:30:00Z', local='2018-03-11T00:00:00+05:30'), datetime_to=Datetime(utc='2018-03-10T19:30:00Z', local='2018-03-11T01:00:00+05:30'))),
 Measurement(period=Period(label='raw', interval='01:00:00', datetime_from=Datetime(utc='2018-03-10T19:30:00Z', local='2018-03-11T01:00:00+05:30'), datetime_to=Datetime(utc='2018-03-10T20:30:00Z', local='2018-03-11T02:00:00+05:30')), value=7.0, parameter=ParameterBase(id=6, name='so2', units='µg/m³', display_

In [102]:
[f.period.interval for f in readings.results]

['01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',
 '01:00:00',

In [57]:
[f.parameter.name+" "+f.parameter.units for f in readings.results]

['so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 µg/m³',
 'so2 

In [54]:
readings

MeasurementsResponse(headers=Headers(x_ratelimit_limit=60, x_ratelimit_remaining=59, x_ratelimit_used=1, x_ratelimit_reset=60), meta=Meta(name='openaq-api', website='/', page=1, limit=1000, found='>1000'), results=[Measurement(period=Period(label='raw', interval='01:00:00', datetime_from=Datetime(utc='2016-10-09T17:45:00Z', local='2016-10-09T23:15:00+05:30'), datetime_to=Datetime(utc='2016-10-09T18:45:00Z', local='2016-10-10T00:15:00+05:30')), value=600.0, parameter=ParameterBase(id=4, name='co', units='µg/m³', display_name=None), coordinates=None, summary=None, coverage=Coverage(expected_count=1, expected_interval='01:00:00', observed_count=1, observed_interval='00:15:00', percent_complete=100.0, percent_coverage=0.0, datetime_from=Datetime(utc='2016-10-09T18:30:00Z', local='2016-10-10T00:00:00+05:30'), datetime_to=Datetime(utc='2016-10-09T18:45:00Z', local='2016-10-10T00:15:00+05:30'))), Measurement(period=Period(label='raw', interval='01:00:00', datetime_from=Datetime(utc='2016-10-0

In [62]:
[f.value for f in readings.results]

[600.0,
 320.0,
 390.0,
 110.0,
 30.0,
 320.0,
 430.0,
 720.0,
 2960.0,
 210.0,
 620.0,
 1080.0,
 780.0,
 470.0,
 450.0,
 540.0,
 260.0,
 750.0,
 500.0,
 540.0,
 670.0,
 380.0,
 690.0,
 510.0,
 790.0,
 250.0,
 320.0,
 760.0,
 360.0,
 360.0,
 160.0,
 350.0,
 130.0,
 180.0,
 240.0,
 390.0,
 290.0,
 580.0,
 690.0,
 510.0,
 370.0,
 160.0,
 170.0,
 50.0,
 90.0,
 470.0,
 240.0,
 320.0,
 610.0,
 670.0,
 700.0,
 440.0,
 530.0,
 590.0,
 240.0,
 270.0,
 170.0,
 50.0,
 340.0,
 420.0,
 440.0,
 440.0,
 1140.0,
 1120.0,
 280.0,
 400.0,
 120.0,
 60.0,
 480.0,
 470.0,
 300.0,
 250.0,
 490.0,
 470.0,
 100.0,
 80.0,
 420.0,
 470.0,
 200.0,
 220.0,
 320.0,
 240.0,
 230.0,
 130.0,
 680.0,
 310.0,
 460.0,
 290.0,
 650.0,
 480.0,
 330.0,
 70.0,
 320.0,
 220.0,
 320.0,
 1020.0,
 400.0,
 390.0,
 380.0,
 230.0,
 820.0,
 360.0,
 570.0,
 640.0,
 420.0,
 540.0,
 190.0,
 170.0,
 520.0,
 160.0,
 520.0,
 490.0,
 640.0,
 790.0,
 490.0,
 720.0,
 700.0,
 680.0,
 340.0,
 770.0,
 970.0,
 400.0,
 820.0,
 610.0,
 510.0,
 8

In [65]:
readings.results[0].period.datetime_to.utc

'2016-10-09T18:45:00Z'

In [60]:
[f.period.datetime_to.utc for f in readings.results]

['2018-03-10T19:30:00Z',
 '2018-03-10T20:30:00Z',
 '2018-03-10T21:30:00Z',
 '2018-03-10T22:30:00Z',
 '2018-03-10T23:30:00Z',
 '2018-03-11T01:30:00Z',
 '2018-03-11T02:30:00Z',
 '2018-03-11T03:30:00Z',
 '2018-03-11T04:30:00Z',
 '2018-03-11T05:30:00Z',
 '2018-03-11T06:30:00Z',
 '2018-03-11T21:30:00Z',
 '2018-03-11T22:30:00Z',
 '2018-03-11T23:30:00Z',
 '2018-03-12T00:30:00Z',
 '2018-03-12T01:30:00Z',
 '2018-03-12T03:30:00Z',
 '2018-03-12T05:30:00Z',
 '2018-03-13T03:30:00Z',
 '2018-03-13T04:30:00Z',
 '2018-03-13T05:30:00Z',
 '2018-03-13T21:30:00Z',
 '2018-03-14T00:30:00Z',
 '2018-03-14T02:30:00Z',
 '2018-03-14T03:30:00Z',
 '2018-03-14T21:30:00Z',
 '2018-03-14T23:30:00Z',
 '2018-03-15T01:30:00Z',
 '2018-03-15T03:30:00Z',
 '2018-03-15T06:30:00Z',
 '2018-03-16T01:30:00Z',
 '2018-03-16T03:30:00Z',
 '2018-03-16T22:30:00Z',
 '2018-03-16T23:30:00Z',
 '2018-03-17T06:30:00Z',
 '2018-03-17T20:30:00Z',
 '2018-03-18T02:30:00Z',
 '2018-03-18T03:30:00Z',
 '2018-03-18T04:30:00Z',
 '2018-03-18T20:30:00Z',


In [58]:
[f.datetime_to for f in readings.results]

AttributeError: 'Measurement' object has no attribute 'datetime_to'

In [104]:
LocationsResponse.results

[Location(id=2592, name='BWSSB Kadabesanahalli, Bengaluru - KSPCB', locality=None, timezone='Asia/Kolkata', country=CountryBase(id=9, code='IN', name='India'), owner=OwnerBase(id=4, name='Unknown Governmental Organization'), provider=ProviderBase(id=168, name='CPCB'), is_mobile=False, is_monitor=True, instruments=[InstrumentBase(id=2, name='Government Monitor')], sensors=[SensorBase(id=5381, name='co µg/m³', parameter=ParameterBase(id=4, name='co', units='µg/m³', display_name='CO mass')), SensorBase(id=5380, name='no2 µg/m³', parameter=ParameterBase(id=5, name='no2', units='µg/m³', display_name='NO₂ mass')), SensorBase(id=5375, name='o3 µg/m³', parameter=ParameterBase(id=3, name='o3', units='µg/m³', display_name='O₃ mass')), SensorBase(id=13898, name='pm25 µg/m³', parameter=ParameterBase(id=2, name='pm25', units='µg/m³', display_name='PM2.5')), SensorBase(id=13888, name='so2 µg/m³', parameter=ParameterBase(id=6, name='so2', units='µg/m³', display_name='SO₂ mass'))], coordinates=Coordin

In [46]:
LocationsResponse.results[0].sensors

[SensorBase(id=5381, name='co µg/m³', parameter=ParameterBase(id=4, name='co', units='µg/m³', display_name='CO mass')),
 SensorBase(id=5380, name='no2 µg/m³', parameter=ParameterBase(id=5, name='no2', units='µg/m³', display_name='NO₂ mass')),
 SensorBase(id=5375, name='o3 µg/m³', parameter=ParameterBase(id=3, name='o3', units='µg/m³', display_name='O₃ mass')),
 SensorBase(id=13898, name='pm25 µg/m³', parameter=ParameterBase(id=2, name='pm25', units='µg/m³', display_name='PM2.5')),
 SensorBase(id=13888, name='so2 µg/m³', parameter=ParameterBase(id=6, name='so2', units='µg/m³', display_name='SO₂ mass'))]

In [10]:
LocationsResponse.results[1].sensors

[SensorBase(id=14637, name='co µg/m³', parameter=ParameterBase(id=4, name='co', units='µg/m³', display_name='CO mass')),
 SensorBase(id=14636, name='no2 µg/m³', parameter=ParameterBase(id=5, name='no2', units='µg/m³', display_name='NO₂ mass')),
 SensorBase(id=14640, name='o3 µg/m³', parameter=ParameterBase(id=3, name='o3', units='µg/m³', display_name='O₃ mass')),
 SensorBase(id=1303004, name='pm10 µg/m³', parameter=ParameterBase(id=1, name='pm10', units='µg/m³', display_name='PM10')),
 SensorBase(id=14641, name='pm25 µg/m³', parameter=ParameterBase(id=2, name='pm25', units='µg/m³', display_name='PM2.5')),
 SensorBase(id=14634, name='so2 µg/m³', parameter=ParameterBase(id=6, name='so2', units='µg/m³', display_name='SO₂ mass'))]

In [11]:
LocationsResponse.results[2].sensors

[SensorBase(id=12237595, name='co ppb', parameter=ParameterBase(id=102, name='co', units='ppb', display_name='CO')),
 SensorBase(id=12237596, name='no ppb', parameter=ParameterBase(id=24, name='no', units='ppb', display_name='NO')),
 SensorBase(id=12237597, name='no2 ppb', parameter=ParameterBase(id=15, name='no2', units='ppb', display_name='NO₂')),
 SensorBase(id=12237598, name='o3 µg/m³', parameter=ParameterBase(id=3, name='o3', units='µg/m³', display_name='O₃ mass')),
 SensorBase(id=12237599, name='pm10 µg/m³', parameter=ParameterBase(id=1, name='pm10', units='µg/m³', display_name='PM10')),
 SensorBase(id=12237600, name='pm25 µg/m³', parameter=ParameterBase(id=2, name='pm25', units='µg/m³', display_name='PM2.5')),
 SensorBase(id=12237601, name='relativehumidity %', parameter=ParameterBase(id=98, name='relativehumidity', units='%', display_name='RH')),
 SensorBase(id=12237602, name='so2 ppb', parameter=ParameterBase(id=101, name='so2', units='ppb', display_name='SO₂')),
 SensorBase(i

In [20]:
type(LocationsResponse.results[2].sensors[0])

openaq.shared.responses.SensorBase

In [33]:
import pandas as pd
useful_dict = {'id':[f.parameter.id for f in LocationsResponse.results[0].sensors],
'pollutant':[f.parameter.display_name for f in LocationsResponse.results[0].sensors],
'units':[f.parameter.units for f in LocationsResponse.results[0].sensors]}
df_0 = pd.DataFrame(useful_dict)

In [31]:
import pandas as pd
useful_dict = {'id':[f.parameter.id for f in LocationsResponse.results[1].sensors],
'pollutant':[f.parameter.display_name for f in LocationsResponse.results[1].sensors],
'units':[f.parameter.units for f in LocationsResponse.results[1].sensors]}
df_1 = pd.DataFrame(useful_dict)

In [32]:
import pandas as pd
useful_dict = {'id':[f.parameter.id for f in LocationsResponse.results[2].sensors],
'pollutant':[f.parameter.display_name for f in LocationsResponse.results[2].sensors],
'units':[f.parameter.units for f in LocationsResponse.results[2].sensors]}
df_2 = pd.DataFrame(useful_dict)

In [35]:
bangalore_df = pd.concat([df_0, df_1, df_2], axis=0)

In [38]:
bangalore_df.drop_duplicates().sort_values('id').to_csv("parameter_ids.csv", index=False)

In [27]:
set([f.parameter.display_name for f in LocationsResponse.results[2].sensors]+[f.parameter.display_name for f in LocationsResponse.results[1].sensors]+[f.parameter.display_name for f in LocationsResponse.results[0].sensors])

{'CO',
 'CO mass',
 'NO',
 'NO₂',
 'NO₂ mass',
 'O₃ mass',
 'PM10',
 'PM2.5',
 'RH',
 'SO₂',
 'SO₂ mass',
 'Temperature (C)'}

In [8]:
len(LocationsResponse.results)

3

In [42]:
Peoria_LocationsResponse = client.locations.list(radius=10000, coordinates=(40.756011635044246, -89.61680835450245))
Peoria_LocationsResponse

LocationsResponse(headers=Headers(x_ratelimit_limit=60, x_ratelimit_remaining=58, x_ratelimit_used=2, x_ratelimit_reset=51), meta=Meta(name='openaq-api', website='/', page=1, limit=100, found=3), results=[Location(id=2246, name='PEORIA_F', locality='Peoria', timezone='America/Chicago', country=CountryBase(id=155, code='US', name='United States'), owner=OwnerBase(id=4, name='Unknown Governmental Organization'), provider=ProviderBase(id=119, name='AirNow'), is_mobile=False, is_monitor=True, instruments=[InstrumentBase(id=2, name='Government Monitor')], sensors=[SensorBase(id=4052, name='o3 ppm', parameter=ParameterBase(id=10, name='o3', units='ppm', display_name='O₃'))], coordinates=Coordinates(latitude=40.6869, longitude=-89.6072), bounds=[-89.6072, 40.6869, -89.6072, 40.6869], distance=7716.20044812, datetime_first=Datetime(utc='2016-03-06T20:00:00Z', local='2016-03-06T14:00:00-06:00'), datetime_last=Datetime(utc='2025-08-31T21:00:00Z', local='2025-08-31T16:00:00-05:00')), Location(id=

In [43]:
Peoria_LocationsResponse.results

[Location(id=2246, name='PEORIA_F', locality='Peoria', timezone='America/Chicago', country=CountryBase(id=155, code='US', name='United States'), owner=OwnerBase(id=4, name='Unknown Governmental Organization'), provider=ProviderBase(id=119, name='AirNow'), is_mobile=False, is_monitor=True, instruments=[InstrumentBase(id=2, name='Government Monitor')], sensors=[SensorBase(id=4052, name='o3 ppm', parameter=ParameterBase(id=10, name='o3', units='ppm', display_name='O₃'))], coordinates=Coordinates(latitude=40.6869, longitude=-89.6072), bounds=[-89.6072, 40.6869, -89.6072, 40.6869], distance=7716.20044812, datetime_first=Datetime(utc='2016-03-06T20:00:00Z', local='2016-03-06T14:00:00-06:00'), datetime_last=Datetime(utc='2025-08-31T21:00:00Z', local='2025-08-31T16:00:00-05:00')),
 Location(id=2247, name='PEORIA_H', locality='Peoria', timezone='America/Chicago', country=CountryBase(id=155, code='US', name='United States'), owner=OwnerBase(id=4, name='Unknown Governmental Organization'), provid

In [17]:
Idahofalls_LocationsResponse.results[1].sensors

[SensorBase(id=25058, name='pm25 µg/m³', parameter=ParameterBase(id=2, name='pm25', units='µg/m³', display_name='PM2.5'))]

## Checking the relationship between the data

In [1]:
import os
from pathlib import Path

In [2]:
blr_data_path = Path("AQ_data")/"Bangalore"

In [7]:
os.listdir(blr_data_path / "Bapuji Nagar, Bengaluru - KSPCB"/'12235235'/'2025')

['06', '12']

In [3]:
os.listdir(blr_data_path)

['Bapuji Nagar, Bengaluru - KSPCB',
 'BTM Layout, Bengaluru - CPCB',
 'BTM Layout, Bengaluru - KSPCB',
 'BWSSB Kadabesanahalli, Bengaluru - CPCB',
 'BWSSB Kadabesanahalli, Bengaluru - KSPCB',
 'City Railway Station - KSPCB',
 'City Railway Station, Bengaluru - KSPCB',
 'Hebbal, Bengaluru - KSPCB',
 'Hombegowda Nagar, Bengaluru - KSPCB',
 'Jayanagar 5th Block, Bengaluru - KSPCB',
 'Kasturi Nagar, Bengaluru - KSPCB',
 'Peenya, Bengaluru - CPCB',
 'Peenya, Bengaluru - KSPCB',
 'RVCE-Mailasandra, Bengaluru - KSPCB',
 'Sanegurava Halli, Bengaluru - KSPCB',
 'SaneguravaHalli - KSPCB',
 'Shivapura_Peenya, Bengaluru - KSPCB',
 'Silk Board, Bengaluru - KSPCB',
 'SiriJaya']

In [24]:
import pandas as pd
all_pollutants = []
sensor_ids = []
for location_name in os.listdir(blr_data_path):
    for sensor_id in os.listdir(blr_data_path / location_name):
        sensor_ids.append(sensor_id)
        pollutants = []
        for year in os.listdir(blr_data_path / location_name / sensor_id):
            for month in os.listdir(blr_data_path / location_name / sensor_id / year):
                pollutant = os.listdir(blr_data_path / location_name / sensor_id / year / month)[0].split(".")[0]
                csv_path = blr_data_path / location_name / sensor_id / year / month / os.listdir(blr_data_path / location_name / sensor_id / year / month)[0]
                df = pd.read_csv(csv_path)
                if df.shape[1] != 10:
                    print(f"{sensor_id}_{year}_{month}_{pollutant} - shape - {df.shape}")

                pollutants.append(pollutant)
                if len(os.listdir(blr_data_path / location_name / sensor_id / year / month))>1:
                    print(f"Anomaly at above pollutant at {month}")
        all_pollutants.append(pollutants[0])
        pollutants = set(pollutants)
        print(f"sensor id: {sensor_id} | {pollutants = }")

sensor id: 12235235 | pollutants = {'co_12235235'}
sensor id: 12235236 | pollutants = {'no_12235236'}
sensor id: 12235237 | pollutants = {'no2_12235237'}
sensor id: 12235238 | pollutants = {'o3_12235238'}
sensor id: 12235239 | pollutants = {'pm10_12235239'}
sensor id: 12235240 | pollutants = {'pm25_12235240'}
sensor id: 12235241 | pollutants = {'relativehumidity_12235241'}
sensor id: 12235242 | pollutants = {'so2_12235242'}
sensor id: 12235243 | pollutants = {'temperature_12235243'}
sensor id: 20153 | pollutants = {'co_20153'}
sensor id: 20164 | pollutants = {'so2_20164'}
sensor id: 20165 | pollutants = {'pm25_20165'}
sensor id: 20166 | pollutants = {'pm10_20166'}
sensor id: 20167 | pollutants = {'o3_20167'}
sensor id: 20169 | pollutants = {'no2_20169'}
sensor id: 12235356 | pollutants = {'co_12235356'}
sensor id: 12235357 | pollutants = {'no_12235357'}
sensor id: 12235358 | pollutants = {'no2_12235358'}
sensor id: 12235359 | pollutants = {'o3_12235359'}
sensor id: 12235360 | pollutant

In [30]:
df

,Timestamp,min,q02,q25,median,q75,q98,max,avg,sd
0,2024-03-08T00:00:00Z,49.587798,49.587798,49.587798,49.587798,49.587798,49.587798,49.587798,49.587798,NaN
1,2024-03-08T00:30:00Z,49.587798,49.587798,49.587798,49.587798,49.587798,49.587798,49.587798,49.587798,NaN
2,2024-03-08T01:00:00Z,37.860120,37.860120,37.860120,37.860120,37.860120,37.860120,37.860120,37.860120,NaN
3,2024-03-08T01:30:00Z,37.860120,37.860120,37.860120,37.860120,37.860120,37.860120,37.860120,37.860120,NaN
4,2024-03-08T02:00:00Z,52.781250,52.781250,52.781250,52.781250,52.781250,52.781250,52.781250,52.781250,NaN
...,...,...,...,...,...,...,...,...,...,...
83,2024-03-09T17:30:00Z,29.294643,29.294643,29.294643,29.294643,29.294643,29.294643,29.294643,29.294643,NaN
84,2024-03-09T18:00:00Z,30.691964,30.691964,30.691964,30.691964,30.691964,30.691964,30.691964,30.691964,NaN
85,2024-03-09T18:30:00Z,30.691964,30.691964,30.691964,30.691964,30.691964,30.691964,30.691964,30.691964,NaN
86,2024-03-09T19:00:00Z,38.260417,38.260417,38.260417,38.260417,38.260417,38.260417,38.260417,38.260417,NaN


In [29]:
df.columns = ['Timestamp'] + [k.split("_")[-1] for k in df.columns[1:]]

In [35]:
#changing the column names and then save it as a csv file

import pandas as pd

for location_name in os.listdir(blr_data_path):
    for sensor_id in os.listdir(blr_data_path / location_name):
        sensor_ids.append(sensor_id)
        pollutants = []
        for year in os.listdir(blr_data_path / location_name / sensor_id):
            for month in os.listdir(blr_data_path / location_name / sensor_id / year):
                pollutant = os.listdir(blr_data_path / location_name / sensor_id / year / month)[0].split(".")[0]
                csv_path = blr_data_path / location_name / sensor_id / year / month / os.listdir(blr_data_path / location_name / sensor_id / year / month)[0]
                df = pd.read_csv(csv_path)
                df.columns = ['Timestamp'] + [k.split("_")[-1] for k in df.columns[1:]]
                df['pollutant'] = pollutant.split("_")[0]
                df.to_csv(csv_path, index=False)
        print(f"{location_name = }| {sensor_id = } | {year = } | {pollutants = }")

location_name = 'Bapuji Nagar, Bengaluru - KSPCB'| sensor_id = '12235235' | year = '2025' | pollutants = []
location_name = 'Bapuji Nagar, Bengaluru - KSPCB'| sensor_id = '12235236' | year = '2025' | pollutants = []
location_name = 'Bapuji Nagar, Bengaluru - KSPCB'| sensor_id = '12235237' | year = '2025' | pollutants = []
location_name = 'Bapuji Nagar, Bengaluru - KSPCB'| sensor_id = '12235238' | year = '2025' | pollutants = []
location_name = 'Bapuji Nagar, Bengaluru - KSPCB'| sensor_id = '12235239' | year = '2025' | pollutants = []
location_name = 'Bapuji Nagar, Bengaluru - KSPCB'| sensor_id = '12235240' | year = '2025' | pollutants = []
location_name = 'Bapuji Nagar, Bengaluru - KSPCB'| sensor_id = '12235241' | year = '2025' | pollutants = []
location_name = 'Bapuji Nagar, Bengaluru - KSPCB'| sensor_id = '12235242' | year = '2025' | pollutants = []
location_name = 'Bapuji Nagar, Bengaluru - KSPCB'| sensor_id = '12235243' | year = '2025' | pollutants = []
location_name = 'Bapuji Naga

In [34]:
pollutant.split("_")[0]

'co'

In [38]:
import json
with open("response_1756734808678.json","r", encoding='utf-8') as f:
    response = json.load(f)

In [39]:
loc_reponse = [k for k in response['results'] if k['id']==412][0]
loc_reponse['coordinates']

{'latitude': 13.0339, 'longitude': 77.51321111}

In [10]:
loc_reponse['sensors']

[{'id': 722,
  'name': 'co Âµg/mÂ³',
  'parameter': {'id': 4,
   'name': 'co',
   'units': 'Âµg/mÂ³',
   'displayName': 'CO mass'}},
 {'id': 1518,
  'name': 'no2 Âµg/mÂ³',
  'parameter': {'id': 5,
   'name': 'no2',
   'units': 'Âµg/mÂ³',
   'displayName': 'NOâ‚‚ mass'}},
 {'id': 13852,
  'name': 'pm25 Âµg/mÂ³',
  'parameter': {'id': 2,
   'name': 'pm25',
   'units': 'Âµg/mÂ³',
   'displayName': 'PM2.5'}},
 {'id': 13853,
  'name': 'so2 Âµg/mÂ³',
  'parameter': {'id': 6,
   'name': 'so2',
   'units': 'Âµg/mÂ³',
   'displayName': 'SOâ‚‚ mass'}}]

In [40]:
sensor_id = 722
[k['parameter']['units'].split(" ")[-1] for k in loc_reponse['sensors'] if k['id']==sensor_id]

['µg/m³']

In [11]:
import pandas as pd

pd.read_csv("parameter_ids.csv")

,id,pollutant,units
0,1,PM10,µg/m³
1,2,PM2.5,µg/m³
2,3,O₃ mass,µg/m³
3,4,CO mass,µg/m³
4,5,NO₂ mass,µg/m³
5,6,SO₂ mass,µg/m³
6,15,NO₂,ppb
7,24,NO,ppb
8,98,RH,%
9,100,Temperature (C),c


In [ ]:
loc_response = response['results']

In [41]:
# THis is for adding more columns to the csv files and then saving it there itself
import pandas as pd
import os
from pathlib import Path

blr_data_path = Path("AQ_data")/"Bangalore"

for location_name in os.listdir(blr_data_path):
    for sensor_id in os.listdir(blr_data_path / location_name):
        for year in os.listdir(blr_data_path / location_name / sensor_id):
            for month in os.listdir(blr_data_path / location_name / sensor_id / year):
                loc_reponse = [k for k in response['results'] if k['name']==location_name][0] #taking relevant location from a bigger file
                csv_path = blr_data_path / location_name / sensor_id / year / month / os.listdir(blr_data_path / location_name / sensor_id / year / month)[0]
                df = pd.read_csv(csv_path)
                df['location_name'] = location_name
                df['latitude'] = loc_reponse['coordinates']['latitude']
                df['longitude'] = loc_reponse['coordinates']['longitude']
                df['unit'] = [k['parameter']['units'] for k in loc_reponse['sensors'] if k['id']==int(sensor_id)][0]
                df['sensor_id'] = sensor_id
                df['city_name'] = "Bangalore"
                df['state'] = "Karnataka"
                df['country'] = "India"
                df.to_csv(csv_path, index=False, encoding='utf-8-sig')
                print(f" changed the file at {csv_path = }")

 changed the file at csv_path = WindowsPath('AQ_data/Bangalore/Bapuji Nagar, Bengaluru - KSPCB/12235235/2025/06/co_12235235.csv')
 changed the file at csv_path = WindowsPath('AQ_data/Bangalore/Bapuji Nagar, Bengaluru - KSPCB/12235235/2025/12/co_12235235.csv')
 changed the file at csv_path = WindowsPath('AQ_data/Bangalore/Bapuji Nagar, Bengaluru - KSPCB/12235236/2025/06/no_12235236.csv')
 changed the file at csv_path = WindowsPath('AQ_data/Bangalore/Bapuji Nagar, Bengaluru - KSPCB/12235236/2025/12/no_12235236.csv')
 changed the file at csv_path = WindowsPath('AQ_data/Bangalore/Bapuji Nagar, Bengaluru - KSPCB/12235237/2025/06/no2_12235237.csv')
 changed the file at csv_path = WindowsPath('AQ_data/Bangalore/Bapuji Nagar, Bengaluru - KSPCB/12235237/2025/12/no2_12235237.csv')
 changed the file at csv_path = WindowsPath('AQ_data/Bangalore/Bapuji Nagar, Bengaluru - KSPCB/12235238/2025/06/o3_12235238.csv')
 changed the file at csv_path = WindowsPath('AQ_data/Bangalore/Bapuji Nagar, Bengaluru -

In [45]:
for k in df.columns:
    print('"'+k+'"', end=" ")

"Timestamp" "min" "q02" "q25" "median" "q75" "q98" "max" "avg" "sd" "pollutant" "location_name" "latitude" "longitude" "unit" "sensor_id" "city_name" "state" "country" 

In [32]:
os.listdir(blr_data_path)[:1]

['Bapuji Nagar, Bengaluru - KSPCB']

In [29]:
[k['parameter']['units'] for k in loc_reponse['sensors'] if k['id']==int(sensor_id)][0]

'ppb'

In [20]:
[k for k in response['results'] if k['name']==location_name][0]

{'id': 6974,
 'name': 'Bapuji Nagar, Bengaluru - KSPCB',
 'locality': None,
 'timezone': 'Asia/Kolkata',
 'country': {'id': 9, 'code': 'IN', 'name': 'India'},
 'owner': {'id': 8528, 'name': 'Karnataka State Pollution Control Board'},
 'provider': {'id': 168, 'name': 'CPCB'},
 'isMobile': False,
 'isMonitor': True,
 'instruments': [{'id': 2, 'name': 'Government Monitor'},
  {'id': 2, 'name': 'Government Monitor'}],
 'sensors': [{'id': 12235235,
   'name': 'co ppb',
   'parameter': {'id': 102,
    'name': 'co',
    'units': 'ppb',
    'displayName': 'CO'}},
  {'id': 20153,
   'name': 'co Âµg/mÂ³',
   'parameter': {'id': 4,
    'name': 'co',
    'units': 'Âµg/mÂ³',
    'displayName': 'CO mass'}},
  {'id': 12235236,
   'name': 'no ppb',
   'parameter': {'id': 24, 'name': 'no', 'units': 'ppb', 'displayName': 'NO'}},
  {'id': 12235237,
   'name': 'no2 ppb',
   'parameter': {'id': 15,
    'name': 'no2',
    'units': 'ppb',
    'displayName': 'NOâ‚‚'}},
  {'id': 20169,
   'name': 'no2 Âµg/mÂ³'